# 5.2.8 量子机器学习模型的隐私性验证测试



- 输出量子近似优化算法（QAOA）的条件数计算结果

In [1]:
from qkappa import *
import numpy as np
testFile('QAOA/qaoa_10.qasm')

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


==========Evaluate largest eigenvalue==========
iter 99/100, 0.98438019, elapsed time: 0.0006s
=========Evaluate smallest eigenvalue==========
iter 99/100, 0.01584363, elapsed time: 0.0007s
Circuit: QAOA/qaoa_10.qasm
Noise configuration: <function depolarize at 0x7fb2a9b49d80>, 0.010000
Total execution time: 16.3294s
Condition Number: 62.130978
(The max/min eigenvalues are: 0.9844, 0.0158)


(Array(62.130978, dtype=float32),
 Array(0.9843802, dtype=float32),
 Array(0.01584363, dtype=float32))

- 输出变分量子本征求解器算法（VQE）的条件数计算结果

In [2]:
testFile('HFVQE/hf_6_0_5.qasm')

==========Evaluate largest eigenvalue==========
iter 11/100, 0.99324316, elapsed time: 0.0001s
=========Evaluate smallest eigenvalue==========
iter 11/100, 0.00676996, elapsed time: 0.0002s
Circuit: HFVQE/hf_6_0_5.qasm
Noise configuration: <function depolarize at 0x7fb2a9b49d80>, 0.010000
Total execution time: 3.6708s
Condition Number: 146.713409
(The max/min eigenvalues are: 0.9932, 0.0068)


(Array(146.71341, dtype=float32),
 Array(0.99324316, dtype=float32),
 Array(0.00676996, dtype=float32))

- 正常输出量子优越性算法的条件数计算结果

In [3]:
testFile('inst/inst_4x4_3_0.qasm')

==========Evaluate largest eigenvalue==========
iter 11/100, 0.98675460, elapsed time: 0.0019s
=========Evaluate smallest eigenvalue==========
iter 11/100, 0.01324511, elapsed time: 0.0022s
Circuit: inst/inst_4x4_3_0.qasm
Noise configuration: <function depolarize at 0x7fb2a9b49d80>, 0.010000
Total execution time: 3.2071s
Condition Number: 74.499565
(The max/min eigenvalues are: 0.9868, 0.0132)


(Array(74.499565, dtype=float32),
 Array(0.9867546, dtype=float32),
 Array(0.01324511, dtype=float32))

- 正确性测试

In [4]:
def answer_gen():
    dic = {}
    dic[('hf_8_0_5.qasm', "dep", 0.01)] = 135.50
    dic[('hf_8_0_5.qasm', "dep", 0.001)] = 1412.58
    dic[('hf_8_0_5.qasm', "bit", 0.01)] = 98.39
    dic[('hf_8_0_5.qasm', "bit", 0.001)] = 991.73

    dic[('hf_10_0_5.qasm', "dep", 0.01)] = 132.21
    dic[('hf_10_0_5.qasm', "dep", 0.001)] = 1432.75
    dic[('hf_10_0_5.qasm', "bit", 0.01)] = 97.64
    dic[('hf_10_0_5.qasm', "bit", 0.001)] = 988.26

    dic[('hf_12_0_5.qasm', "dep", 0.01)] = 140.58
    dic[('hf_12_0_5.qasm', "dep", 0.001)] = 1438.94
    dic[('hf_12_0_5.qasm', "bit", 0.01)] = 95.27
    dic[('hf_12_0_5.qasm', "bit", 0.001)] = 978.87

    return dic



In [5]:
def dp_test(epsilon, eta, kappa):
    epsilon_star = np.log((kappa-1)*eta+1)
    return epsilon>epsilon_star

eta = 1
epsilon = 5

def validation():
    for file in os.listdir('Test_Benchmarks/'):
        k_test = testFile('Test_Benchmarks/'+file, cirq.depolarize, 0.01)[0]
        k_truth = answer_gen()[(file, "dep", 0.01)]
        assert(dp_test(epsilon, eta, k_test) == dp_test(epsilon, eta, k_truth))
        k_test = testFile('Test_Benchmarks/'+file, cirq.depolarize, 0.001)[0]
        k_truth = answer_gen()[(file, "dep", 0.001)]
        assert(dp_test(epsilon, eta, k_test) == dp_test(epsilon, eta, k_truth))
        k_test = testFile('Test_Benchmarks/'+file, cirq.bit_flip, 0.01)[0]
        k_truth = answer_gen()[(file, "bit", 0.01)]
        assert(dp_test(epsilon, eta, k_test) == dp_test(epsilon, eta, k_truth))
        k_test = testFile('Test_Benchmarks/'+file, cirq.bit_flip, 0.001)[0]
        k_truth = answer_gen()[(file, "bit", 0.001)]
        assert(dp_test(epsilon, eta, k_test) == dp_test(epsilon, eta, k_truth))
    print("Validation Passed")


In [6]:
validation()

==========Evaluate largest eigenvalue==========
iter 99/100, 0.99207377, elapsed time: 0.0021s
=========Evaluate smallest eigenvalue==========
iter 99/100, 0.00784659, elapsed time: 0.0008s
Circuit: Test_Benchmarks/hf_8_0_5.qasm
Noise configuration: <function depolarize at 0x7fb2a9b49d80>, 0.010000
Total execution time: 18.4783s
Condition Number: 126.433685
(The max/min eigenvalues are: 0.9921, 0.0078)
==========Evaluate largest eigenvalue==========
iter 11/100, 0.99916971, elapsed time: 0.0022s
=========Evaluate smallest eigenvalue==========
iter 11/100, 0.00085342, elapsed time: 0.0011s
Circuit: Test_Benchmarks/hf_8_0_5.qasm
Noise configuration: <function depolarize at 0x7fb2a9b49d80>, 0.001000
Total execution time: 5.2232s
Condition Number: 1170.784058
(The max/min eigenvalues are: 0.9992, 0.0009)
==========Evaluate largest eigenvalue==========
iter 99/100, 0.98901302, elapsed time: 0.0012s
=========Evaluate smallest eigenvalue==========
iter 99/100, 0.01113844, elapsed time: 0.0017